# Standardized Data With Timestamps Suitable For Backtesting

### [Data Dictionary](https://www.calcbench.com/home/standardizedmetrics)
### [Documentation](http://calcbench.github.io/python_api_client/html/numeric-data.html#point-in-time)

In [ ]:
%pip install calcbench-api-client[Pandas,Backoff] tqdm

In [ ]:
from datetime import date, timedelta

from pathlib import Path
from tqdm import tqdm
import pandas as pd

import calcbench as cb
from calcbench.downloaders import iterate_and_save_pandas

In [ ]:
cb.turn_on_logging()

## Historic Data For Individual Companies

In [ ]:
tickers = ["MSFT", "AMZN", "XOM"] # cb.tickers(entire_universe=True) 

In [ ]:
face_output_file_name = Path.joinpath(Path.home(), "cb_pit_data.csv")

iterate_and_save_pandas(
    tickers,
    lambda ticker: cb.standardized(company_identifiers=[ticker], point_in_time=True),
    face_output_file_name,
)

## Push Notification of New Data
### Example @ https://github.com/calcbench/notebooks/tree/master/filing_listener

## Get All Data for One Day (Yesterday)

In [ ]:
start_date = date.today() - timedelta(days=1)
end_date = date.today()
face_output_file_name = Path.joinpath(Path.home(), f"cb_pit_data_{start_date}.csv")
d = cb.standardized(
    start_date=start_date,
    end_date=end_date,
    point_in_time=True,
)
#d.reset_index().to_csv(face_output_file_name)